Installing FFMPEG

In [5]:
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

Installation finished.


Locating Video

In [6]:
%cd /content/
!mkdir videos_to_infer
!mkdir inferred_videos
%cd videos_to_infer

/content
mkdir: cannot create directory ‘videos_to_infer’: File exists
mkdir: cannot create directory ‘inferred_videos’: File exists
/content/videos_to_infer


Break down video frames into images

In [7]:
# break video down into images - UPDATE THE PATH TO THE FILE!
# Example: video file named 'Test.mp4'
# (cont'd) update path to: ""
os.environ['inputFile'] = '/content/videos_to_infer/Test.mp4'

# fps value: the number of frames to sample per second from the video
# higher value for fps: sample more frames
!ffmpeg  -hide_banner -loglevel error -i "$inputFile" -vf fps=30 "$inputFile_out%04d.png" 

Install libraries

In [8]:
!pip3 install roboflow
!pip3 install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=8ef705853ba4057a671ecce76617b00f6758e2f08cf7e6c68823e5a5edebaccb
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 15.0 MB/s eta 0:00:00


Roboflow Model

In [9]:
# workspace code
from roboflow import Roboflow
import json

rf = Roboflow(api_key="O4zcgkVthFhFXjvCzJwx")
project = rf.workspace("beni-suef-university-casvi").project("collision-detection")
dataset = project.version(2).download("yolov8")

# grab the model from that project's version
model = project.version(2).model
print(model)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.106, to fix: `pip install ultralytics<=8.0.20`

Extracting Dataset Version Zip to collision-detection-2 in yolov8:: 100%|██████████| 3176/3176 [00:01<00:00, 1995.96it/s]


{
  "id": "collision-detection/2",
  "name": "collision detection",
  "version": "2",
  "classes": null,
  "overlap": 30,
  "confidence": 40,
  "stroke": 1,
  "labels": false,
  "format": "json",
  "base_url": "https://detect.roboflow.com/"
}


Imports

In [10]:
from roboflow import Roboflow
import json
from time import sleep
from PIL import Image, ImageDraw
import io
import base64
import requests
from os.path import exists
import os, sys, re, glob

In [18]:
# HELPER FUNCTIONS BLOCK
def draw_boxes(box, x0, y0, img, class_name):
    # OPTIONAL - color map, change the key-values for each color to make the
    # class output labels specific to your dataset
    color_map = {
        'Antenna': 'red',
        'Camera': 'blue',
        'Earth': 'green',
        'MainBody': 'yellow',
        'Moon': 'purple',
        'Orbit': 'orange',
        'Planet': 'pink',
        'Rocket': 'cyan',
        'Satellite': 'magenta',
        'Solar Cell': 'brown',
        'Space': 'gray',
        'Space_Object': 'lime',
        'Spacecraft': 'teal',
        'Sun': 'olive',
        'space': 'navy'
    }
    # get position coordinates
    bbox = ImageDraw.Draw(img) 

    bbox.rectangle(box, outline =color_map[class_name], width=5)
    bbox.text((x0, y0), class_name, fill='white', anchor='mm')

    return img

def save_with_bbox_renders(img):
    file_name = os.path.basename(img.filename)
    img.save('/content/inferred_videos/' + file_name)

Excution

In [ ]:
# perform inference on each image from the split up video

%cd /content/inferred_videos/
!pwd
# glob config values
file_path = "/content/videos_to_infer/"
extention = ".png"

# glob files based on location and file format
globbed_files = sorted(glob.glob(file_path + '*' + extention))
print(globbed_files)

for image in globbed_files:
  # INFERENCE
  predictions = model.predict(image).json()['predictions']
  newly_rendered_image = Image.open(image)

  # RENDER 
  # for each detection, create a crop and convert into CLIP encoding
  print(predictions)
  for prediction in predictions:
      # rip bounding box coordinates from current detection
      # note: infer returns center points of box as (x,y) and width, height
      # ----- but pillow crop requires the top left and bottom right points to crop
      x0 = prediction['x'] - prediction['width'] / 2
      x1 = prediction['x'] + prediction['width'] / 2
      y0 = prediction['y'] - prediction['height'] / 2
      y1 = prediction['y'] + prediction['height'] / 2
      box = (x0, y0, x1, y1)
  
      newly_rendered_image = draw_boxes(box, x0, y0, newly_rendered_image, prediction['class'])

  # WRITE
  save_with_bbox_renders(newly_rendered_image)

Kalman filters

In [44]:
import cv2
import numpy as np
import glob
import os

class KalmanFilter:
    def __init__(self):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def predict(self, coordX, coordY):
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y

frame_files = sorted(glob.glob("/content/inferred_videos/*.png"))  # Update glob pattern here
kf = KalmanFilter()

output_directory = "output_frames"
os.makedirs(output_directory, exist_ok=True)

classes_to_filter = ['Antenna', 'Camera', 'MainBody', 'Rocket', 'Satellite', 'Solar Cell', 'Space_Object', 'Spacecraft']

color_map = {
    'Antenna': (0, 0, 255),        # Red
    'Camera': (255, 0, 0),        # Blue
    'MainBody': (0, 255, 255),    # Yellow
    'Rocket': (255, 255, 0),      # Cyan
    'Satellite': (255, 0, 255),   # Magenta
    'Solar Cell': (42, 42, 165),  # Brown
    'Space_Object': (0, 255, 0),  # Lime
    'Spacecraft': (0, 128, 128),  # Teal
}

prev_cx, prev_cy = None, None  # Store previous position


for frame_file in frame_files:
    frame = cv2.imread(frame_file)

    # Assuming you have obtained the bounding box coordinates (x, y, x2, y2) and class name from the detection
    # Modify the code below based on your detection implementation
    predictions = model.predict(frame_file).json()['predictions']
    for prediction in predictions:
        class_name = prediction['class']
        x0 = prediction['x']
        y0 = prediction['y']
        x1 = prediction['x']
        y1 = prediction['y']

        # Check if the class is in the classes_to_filter list
        # Check if the class is in the color_map dictionary
        if class_name in classes_to_filter and class_name in color_map:
            cx = int((x0 + x1) / 2)
            cy = int((y0 + y1) / 2)

            if prev_cx is not None and prev_cy is not None:
                # Compare current position with previous position
                dx = cx - prev_cx
                dy = cy - prev_cy

                if dx > 0:
                    direction = "Moving towards me"
                elif dx < 0:
                    direction = "Moving away from me"
                else:
                    direction = "Not moving"

                print("Object:", class_name)
                print("Direction:", direction)

            prev_cx, prev_cy = cx, cy

            predicted = kf.predict(cx, cy)
            cv2.circle(frame, (cx, cy), 10, color_map[class_name], -1)
            cv2.circle(frame, (predicted[0], predicted[1]), 10, (255, 0, 0), 4)
    # Save the modified frame as PNG in the output directory
    frame_name = os.path.basename(frame_file)
    output_frame_path = os.path.join(output_directory, frame_name)
    cv2.imwrite(output_frame_path, frame)

print("Output frames saved in", output_directory)



Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving towards
Object: Satellite
Direction: Moving away from
Object: Satellite
Direction: Moving towards
Object: Antenna
Direction: Moving away from
Object: Space_Object
Direction: Not moving
Object: Solar Cell
Direction: Moving towards
Object: Satellite
Direction: Moving towards
Object: Solar Cell

Bring back Video

In [48]:
import subprocess

%cd /content/
!mkdir Result

input_path = "/content/output_frames/%04d.png"
output_path = "/content/Result/Result.mp4"

# Execute FFmpeg command
command = [
    "ffmpeg",
    "-r", "30",
    "-s", "1920x1080",
    "-i", input_path,
    "-vcodec", "libx264",
    "-crf", "15",
    "-pix_fmt", "yuv420p",
    output_path
]

subprocess.run(command)

print("Output Result video saved in:", output_path)

/content
Output Result video saved in: /content/Result/Result.mp4


Fresh Start

In [49]:
# CAUTION - deletes all data from the `inferred_videos` and `videos_to_infer` directories
# use this code block between runs for a fresh start between videos.
%cd /content/
!rm -r inferred_videos
!rm -r videos_to_infer
!rm -r output_frames
!rm -r Result
!mkdir inferred_videos
!mkdir videos_to_infer

/content
